# Transformers能做什么？

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

## 安装


In [ ]:
# compatitable versions of the dependencies
# cell magic command "%%"

In [19]:
%%writefile requirements.txt
datasets==2.13.0
fsspec==2023.6.0
gcsfs==2023.6.0
evaluate
transformers[sentencepiece]

Overwriting requirements.txt


In [20]:
# !pip uninstall -y datasets evaluate transformers gcsfs fsspec
!pip install -r requirements.txt

  Using cached datasets-2.13.0-py3-none-any.whl.metadata (20 kB)
  Using cached fsspec-2023.6.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached gcsfs-2023.6.0-py2.py3-none-any.whl.metadata (1.6 kB)
  Using cached evaluate-0.4.3-py3-none-any.whl.metadata (9.2 kB)
  Using cached transformers-4.48.0-py3-none-any.whl.metadata (44 kB)
Using cached datasets-2.13.0-py3-none-any.whl (485 kB)
Using cached fsspec-2023.6.0-py3-none-any.whl (163 kB)
Using cached gcsfs-2023.6.0-py2.py3-none-any.whl (26 kB)
Using cached evaluate-0.4.3-py3-none-any.whl (84 kB)
Using cached transformers-4.48.0-py3-none-any.whl (9.7 MB)


## pipeline函数

- 高层次的接口，封装了模型加载、数据预处理和推理的过程
- 操作：
  - 指定任务；下载model&tokenizer；
  - build pipeline：data -> tokenizer -> model(deduction) -> return results
  - 其实就是3步：文本被预处理为模型可以理解的格式；预处理的输入被传递给模型；模型处理后输出最终人类可以理解的结果。

In [4]:
# 检查gpu是否启用&是否分配任务
import torch
print(torch.cuda.is_available())
!nvidia-smi

True
Sun Jan 12 14:31:22 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0              27W /  70W |    421MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+--------------------

In [8]:
"""
可以指定参数，task, model, tokenizer, device...
指定任务类型，常见任务如下：
	•	"sentiment-analysis"：情感分析。
	•	"text-classification"：文本分类。
	•	"question-answering"：问答系统。
	•	"translation_xx_to_yy"：机器翻译。
	•	"summarization"：文本摘要。
	•	"text-generation"：文本生成。
	•	"ner"：命名实体识别。
	•	"fill-mask"：掩码填充（如 BERT 的 MASK）。
  • 特征提取，零样本分类...
"""
from transformers import pipeline
# load default model&tokenizer
classifier = pipeline(task="sentiment-analysis", \
                      model="distilbert-base-uncased-finetuned-sst-2-english",\
                      device=0)
classifier("I've been waiting for a HuggingFace course my whole life.")

Device set to use cuda:0


[{'label': 'POSITIVE', 'score': 0.9598046541213989}]

In [9]:
classifier(
    ["I've been waiting for a HuggingFace course my whole life.", "I hate this so much!"]
)

[{'label': 'POSITIVE', 'score': 0.9598046541213989},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

In [13]:
# 再例如零样本分类任务
classifier = pipeline("zero-shot-classification")
# classifier(
#     "This is a course about the Transformers library",
#     candidate_labels=["education", "politics", "business"],
# )
classifier(
    "University in Germany", \
    candidate_labels=["course", "shop", "cat", "light", "professor", "teacher"],
)

No model was supplied, defaulted to facebook/bart-large-mnli and revision d7645e1 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0


{'sequence': 'University in Germany',
 'labels': ['course', 'professor', 'light', 'teacher', 'cat', 'shop'],
 'scores': [0.635637104511261,
  0.12215486168861389,
  0.11887802183628082,
  0.08362175524234772,
  0.024133972823619843,
  0.015574224293231964]}

In [14]:
from transformers import pipeline

generator = pipeline("text-generation")
generator("In this course, we will teach you how to")

No model was supplied, defaulted to openai-community/gpt2 and revision 607a30d (https://huggingface.co/openai-community/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, we will teach you how to construct a simple example that shows how to create something with a simple name. Once you know how to do this, you should be ready to publish your first prototype onto the market.\n\nIn the'}]

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="distilgpt2")
generator(
    "In this course, we will teach you how to",
    max_length=30,
    num_return_sequences=2,
)

[{'generated_text': 'In this course, we will teach you how to manipulate the world and '
                    'move your mental and physical capabilities to your advantage.'},
 {'generated_text': 'In this course, we will teach you how to become an expert and '
                    'practice realtime, and with a hands on experience on both real '
                    'time and real'}]

In [ ]:
from transformers import pipeline

unmasker = pipeline("fill-mask")
unmasker("This course will teach you all about <mask> models.", top_k=2)

[{'sequence': 'This course will teach you all about mathematical models.',
  'score': 0.19619831442832947,
  'token': 30412,
  'token_str': ' mathematical'},
 {'sequence': 'This course will teach you all about computational models.',
  'score': 0.04052725434303284,
  'token': 38163,
  'token_str': ' computational'}]

In [ ]:
from transformers import pipeline

ner = pipeline("ner", grouped_entities=True)
ner("My name is Sylvain and I work at Hugging Face in Brooklyn.")

[{'entity_group': 'PER', 'score': 0.99816, 'word': 'Sylvain', 'start': 11, 'end': 18}, 
 {'entity_group': 'ORG', 'score': 0.97960, 'word': 'Hugging Face', 'start': 33, 'end': 45}, 
 {'entity_group': 'LOC', 'score': 0.99321, 'word': 'Brooklyn', 'start': 49, 'end': 57}
]

In [ ]:
from transformers import pipeline

question_answerer = pipeline("question-answering")
question_answerer(
    question="Where do I work?",
    context="My name is Sylvain and I work at Hugging Face in Brooklyn",
)

{'score': 0.6385916471481323, 'start': 33, 'end': 45, 'answer': 'Hugging Face'}
klyn",
)


In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization")
summarizer(
    """
    America has changed dramatically during recent years. Not only has the number of
    graduates in traditional engineering disciplines such as mechanical, civil,
    electrical, chemical, and aeronautical engineering declined, but in most of
    the premier American universities engineering curricula now concentrate on
    and encourage largely the study of engineering science. As a result, there
    are declining offerings in engineering subjects dealing with infrastructure,
    the environment, and related issues, and greater concentration on high
    technology subjects, largely supporting increasingly complex scientific
    developments. While the latter is important, it should not be at the expense
    of more traditional engineering.

    Rapidly developing economies such as China and India, as well as other
    industrial countries in Europe and Asia, continue to encourage and advance
    the teaching of engineering. Both China and India, respectively, graduate
    six and eight times as many traditional engineers as does the United States.
    Other industrial countries at minimum maintain their output, while America
    suffers an increasingly serious decline in the number of engineering graduates
    and a lack of well-educated engineers.
"""
)

[{'summary_text': ' America has changed dramatically during recent years . The '
                  'number of engineering graduates in the U.S. has declined in '
                  'traditional engineering disciplines such as mechanical, civil '
                  ', electrical, chemical, and aeronautical engineering . Rapidly '
                  'developing economies such as China and India, as well as other '
                  'industrial countries in Europe and Asia, continue to encourage '
                  'and advance engineering .'}]

In [ ]:
from transformers import pipeline

translator = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en")
translator("Ce cours est produit par Hugging Face.")

[{'translation_text': 'This course is produced by Hugging Face.'}]


## 到git

In [17]:
# !git --version
!git clone https://github.com/SonyaX20/ai_check.git

Cloning into 'ai_check'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.
